In [1]:
from postgres_connect import *
import requests

In [2]:
query = '''
select r.rider_id, node_node_id
from rider_pip 
left join application_db.rider as r on r.locus_rider_id = rider_pip.rider_id
where 
        pip_status = 'closed' 
and     pip_result = 'failed'
order by node_node_id
'''

In [6]:
path = '/home/akshay/Documents/tunnel-ssh .cer'
# printthis('yes')
conn = get_conn('Yes',path)
df_block = pd.read_sql(query,conn)
conn.close()

Tunnel Started
Connection Made


In [7]:
df_block

,rider_id,node_node_id
0,3565,2
1,2709,2
2,3264,2
3,3512,2
4,3399,3
5,3581,3
6,3618,3
7,3532,3
8,3677,3
9,3688,3


In [ ]:
df_block.to_csv('block_list_18_jan.csv')

In [9]:
url = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9001/rider/delist'
headers_base = {
    'name': 'void',  # Change this value if needed
    'Content-Type': 'application/json',
}

for index, row in df_block.iterrows():
    headers = headers_base.copy()
    headers['node_id'] = str(row['node_node_id'])

    data = {
        'delistType': 'BLACKLIST',
        'riderIds': [str(row['rider_id'])],
    }

    response = requests.put(url, headers=headers, json=data)

    try:
        json_response = response.json()
        message = json_response.get('message', '')
        print(f"Request for rider_id {row['rider_id']} - {message}")

        data = json_response.get('data', [])
        for item in data:
            rider_id = item.get('riderId', '')
            rider_name = item.get('riderName', '')
            error_message = item.get('message', '')
            print(f"    Rider ID: {rider_id}, Rider Name: {rider_name}, Error Message: {error_message}")

    except ValueError:
        print("Invalid JSON response")
    
print("Rider blocking completed updating database")


KeyboardInterrupt: 

In [10]:

import requests
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Constants
BASE_URL = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9001'
API_ENDPOINT = '/rider/delist'
HEADERS_BASE = {
    'name': 'void',  # Change this value if needed
    'Content-Type': 'application/json',
}

# Create a session for making multiple requests
session = requests.Session()

# List to store rider IDs with timeout
timeout_rider_ids = []

for index, row in df_block.iterrows():
    headers = HEADERS_BASE.copy()
    headers['node_id'] = str(row['node_node_id'])

    data = {
        'delistType': 'BLACKLIST',
        'riderIds': [str(row['rider_id'])],
    }

    url = f"{BASE_URL}{API_ENDPOINT}"

    try:
        response = session.put(url, headers=headers, json=data, timeout=5)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        json_response = response.json()
        message = json_response.get('message', '')
        logging.info(f"Request for rider_id {row['rider_id']} - {message}")

        data = json_response.get('data', [])
        for item in data:
            rider_id = item.get('riderId', '')
            rider_name = item.get('riderName', '')
            error_message = item.get('message', '')
            logging.info(f"    Rider ID: {rider_id}, Rider Name: {rider_name}, Error Message: {error_message}")

    except requests.exceptions.Timeout:
        logging.error(f"Timeout occurred for rider_id {row['rider_id']}")
        timeout_rider_ids.append(row['rider_id'])

    except requests.exceptions.HTTPError as errh:
        logging.error(f"HTTP Error: {errh}")
    except requests.exceptions.RequestException as err:
        logging.error(f"Request Exception: {err}")

print("Rider blocking completed updating database")

# Print or use timeout_rider_ids as needed
print("Rider IDs with timeout:", timeout_rider_ids)


ERROR:root:Timeout occurred for rider_id 3565
ERROR:root:Timeout occurred for rider_id 2709
ERROR:root:Timeout occurred for rider_id 3264
ERROR:root:Timeout occurred for rider_id 3512
ERROR:root:Timeout occurred for rider_id 3399
ERROR:root:Timeout occurred for rider_id 3581
ERROR:root:Timeout occurred for rider_id 3618


KeyboardInterrupt: 

In [32]:
conn = get_conn('Yes',path)
cursor = conn.cursor()
update_query1 = '''
    UPDATE rider_pip
    SET pip_status = 'final'
    WHERE pip_status = 'closed';
'''
update_query2 = '''
delete from rider_pip where pip_status = 'running' and rider_id in 
(select rider_id from rider_pip where pip_status = 'final' and pip_result = 'failed')
'''
cursor.execute(update_query1)
print("Updated all closed status to final")
cursor.execute(update_query2)
print("Deleted blocked riders from running pip")
conn.commit()
cursor.close()
conn.close()
print("Updated all closed status to final")

Tunnel Started
Connection Made
Updated all closed status to final
Deleted blocked riders from running pip
Updated all closed status to final
